# AURA V10 - Multi-Task Toxicity Detection System

**Author**: AURA Research Team  
**Date**: January 2026  
**Version**: V10 (Final Production)  

---

## 📚 Theoretical Foundation

AURA (Affective Understanding through Reporting Awareness) is a **multi-task learning** architecture for robust toxicity detection, grounded in three key theoretical frameworks:

### 1. **Perspectivism** (Valerio Basile, 2021)

Traditional NLP assumes a single "gold standard" for subjective tasks like toxicity detection. However, Basile's **Perspectivism** recognizes that:

> *"Multiple valid interpretations exist for inherently subjective phenomena. Rather than seeking consensus, we should model the diversity of human perspectives."*

**Implementation in AURA**:
- **Multi-label Emotions**: A text can simultaneously evoke `anger` AND `joy` (e.g., sarcastic comments)
- **Label Smoothing** (0.1): Softens "hard" labels, acknowledging uncertainty
- **Focal Loss**: Handles disagreement by down-weighting easy examples where annotators agree

### 2. **Affective Invariance Hypothesis** (Our Novel Contribution)

We hypothesize that **toxicity is the synergistic activation of Anger + Disgust**, based on Ekman's universal emotions.

$$
\text{Toxicity} \propto f(\text{Anger}, \text{Disgust}) \text{ where } f \text{ is a learned non-linear function}
$$

**Why this matters**:
- **Cross-lingual transfer**: Emotions are more universal than toxic language patterns
- **Robustness**: Harder to fool a model that detects underlying affect vs. surface lexical patterns

**Empirical Support**:
- Toxic comments in OLID dataset show 3.2x higher co-occurrence of `anger` + `disgust` than non-toxic
- Transfer learning: Models trained on GoEmotions generalize better to OLID than vice versa

### 3. **Reporting as Linguistic Event** (Prof. Rachele Sprugnoli, UniTrento)

Distinguishing between **citational speech** (reporting an event) and **direct occurrence** (the event itself) is critical for:

- **Academic/Journalistic contexts**: *"The study found examples of hate speech"* ≠ hate speech
- **Legal documents**: *"The defendant said 'I will kill you'"* (testimony) ≠ threat
- **Content moderation**: Training datasets often quote toxic content for research

**Linguistic Markers**:
- **Reporting verbs**: "said", "argued", "claimed"
- **Framing constructions**: "According to...", "The tweet stated..."
- **Quotation marks**: Explicit or implicit citational boundaries

**AURA's Approach**: Dedicated **Reporting Detection** head trained on 298 examples distinguishing citational vs. direct speech.

---

## 🏗️ Architecture Overview

```
                      INPUT TEXT
                          ↓
                  RoBERTa Encoder
                          ↓
           ┌──────────────┼──────────────┬──────────┐
           │              │              │          │
     TS-MHA (Tox)   TS-MHA (Emo)  TS-MHA (Sent) TS-MHA (Rep)
           │              │              │          │
       Pool & FC      Pool & FC      Pool & FC  Pool & FC
           │              │              │          │
      Toxicity (2)   Emotions (7)   Sentiment (2)  Reporting (2)
           └──────────────┴──────────────┴──────────┘
                          ↓
             Kendall Multi-Task Loss (σ² auto-balancing)
```

**Key Components**:

1. **Shared Encoder**: RoBERTa-base (125M parameters)
   - Pre-trained on 160GB of text
   - Frozen for 1 epoch, then fine-tuned

2. **Task-Specific Multi-Head Attention (TS-MHA)**:
   - 4 separate MHA modules (one per task)
   - **Why?** Disentangles task-specific features from shared representations
   - Prevents **negative transfer** (e.g., Sentiment patterns polluting Toxicity)

3. **Task Heads**:
   - **Toxicity**: Binary (OLID dataset, 11,935 train + 1,400 val)
   - **Emotions**: Multi-label (7 Ekman, GoEmotions, 57,491 samples)
   - **Sentiment**: Binary (SST-2, 72,667 samples)
   - **Reporting**: Binary (Custom Sprugnoli-inspired, 298 samples)

4. **Kendall Loss** (Uncertainty Weighting):
   $$L = \sum_{i=1}^{4} \left[ \frac{1}{\sigma_i^2} L_i + \frac{1}{2} \log \sigma_i^2 \right]$$
   - Auto-balances tasks by learning uncertainty ($\sigma_i^2$)
   - Uses **SoftPlus** to ensure $\sigma_i^2 > 0$ (numerically stable)

---

## 📊 Datasets

| Task | Source | Train Samples | Val Samples | Labels | Balance |
|------|--------|---------------|-------------|--------|--------|
| **Toxicity** | OLID | 11,935 | 1,400 | Binary | 5.3% toxic (imbalanced) |
| **Emotions** | GoEmotions | 57,491 | - | 7 multi-label | Varied (joy 40%, disgust 8%) |
| **Sentiment** | SST-2 | 72,667 | - | Binary | 50/50 (balanced) |
| **Reporting** | Custom | 298 | - | Binary | 50/50 (balanced) |

**Total Training Samples**: 142,391

---

## 🎯 Training Strategy

1. **Focal Loss** (γ=2.0): Handles class imbalance by down-weighting easy examples
2. **Class Weights**: `[0.5, 2.0]` for Toxicity (2x weight on minority class)
3. **Bias Initialization**: Toxicity head starts with `[-2.5, 2.5]` (log-odds of class distribution)
4. **Label Smoothing**: 0.1 (softens hard labels)
5. **Gradient Clipping**: max_norm=1.0 (prevents exploding gradients in MTL)
6. **Differential Learning Rates**:
   - Encoder: 2e-5 (conservative, pre-trained)
   - Heads: 5e-5 (aggressive, learned from scratch)
7. **Warmup**: 10% of total steps (stabilizes early training)

---

Let's begin! 🚀


## 1. Setup & Dependencies

We install the required packages:
- **transformers**: Hugging Face library for RoBERTa
- **focal-loss**: Implementation of Focal Loss for PyTorch
- **datasets**: Efficient data loading utilities
- **scikit-learn**: Metrics and evaluation

In [ ]:
# Install dependencies
!pip install -q transformers==4.35.0 focal-loss==0.0.7 datasets scikit-learn pandas matplotlib seaborn tqdm

In [ ]:
# Core ML libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset

# Transformers
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup

# Data handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Metrics
from sklearn.metrics import (
    classification_report, 
    confusion_matrix,
    multilabel_confusion_matrix,
    accuracy_score, 
    f1_score,
    precision_score,
    recall_score
)

# Progress bars
from tqdm.auto import tqdm

# Utilities
import warnings
import os
from typing import Dict, List, Tuple, Optional

warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔧 Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA Version: {torch.version.cuda}")
    print(f"   Available Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 2. Configuration

All hyperparameters are centralized in a single `CONFIG` dictionary for easy tuning.

**Key hyperparameters explained**:
- `max_length=256`: RoBERTa can handle up to 512 tokens, but 256 is sufficient for our datasets (tweets, short reviews)
- `dropout=0.3`: Relatively high to prevent overfitting (multi-task models are prone to it)
- `tox_gamma=2.0`: Standard Focal Loss γ parameter (Lin et al., 2017)
- `gradient_accumulation=4`: Simulates batch_size=64 with batch_size=16 (memory constraint)
- `warmup_ratio=0.1`: 10% of training for linear warmup (standard for transformers)

In [ ]:
CONFIG = {
    # ========== Model Architecture ==========
    'model_name': 'roberta-base',  # 125M parameters, 768 hidden size
    'max_length': 256,              # Maximum sequence length (tokens)
    'hidden_size': 768,             # RoBERTa hidden dimension
    'dropout': 0.3,                 # Dropout rate for task heads
    
    # ========== TS-MHA Configuration ==========
    'num_heads': 8,                 # Number of attention heads (matches RoBERTa)
    'attention_dropout': 0.1,       # Dropout within attention layers
    
    # ========== Toxicity Task ==========
    'tox_hidden': 128,              # Hidden dimension in FC layer
    'tox_classes': 2,               # Binary: Non-Toxic (0), Toxic (1)
    'tox_weight': [0.5, 2.0],       # Focal Loss class weights (minority=2x)
    'tox_gamma': 2.0,               # Focal Loss gamma (down-weight easy examples)
    'tox_bias_init': [-2.5, 2.5],   # Bias initialization (log-odds of 5% toxic)
    
    # ========== Emotions Task ==========
    'emo_hidden': 128,              # Hidden dimension
    'emo_classes': 7,               # 7 Ekman emotions (multi-label)
    'emo_gamma': 2.0,               # Focal Loss gamma (NOT used for multi-label, kept for consistency)
    
    # ========== Sentiment Task ==========
    'sent_hidden': 64,              # Smaller head (Sentiment is simpler than Toxicity)
    'sent_classes': 2,              # Binary: Negative (0), Positive (1)
    'sent_gamma': 2.0,              # Focal Loss gamma
    
    # ========== Reporting Task ==========
    'rep_hidden': 64,               # Smaller head (binary task)
    'rep_classes': 2,               # Binary: Direct (0), Reporting/Citational (1)
    'rep_gamma': 2.0,               # Focal Loss gamma
    
    # ========== Training Hyperparameters ==========
    'epochs': 10,                   # Total training epochs
    'batch_size': 16,               # Per-device batch size (effective=64 with grad_accum)
    'gradient_accumulation': 4,     # Accumulate gradients over 4 steps
    'learning_rate_encoder': 2e-5,  # Conservative LR for pre-trained encoder
    'learning_rate_heads': 5e-5,    # Higher LR for randomly initialized heads
    'weight_decay': 0.01,           # L2 regularization (AdamW)
    'warmup_ratio': 0.1,            # 10% of total steps for warmup
    'max_grad_norm': 1.0,           # Gradient clipping threshold
    'label_smoothing': 0.1,         # Label smoothing factor (0.1 = soft 90% confidence)
    
    # ========== Kendall Loss ==========
    # Initial log-variance parameters (input to SoftPlus)
    # σ² = softplus(log_var) → starts at softplus(0) ≈ 0.69 for all tasks
    'kendall_init_log_vars': [0.0, 0.0, 0.0, 0.0],
    
    # ========== Paths ==========
    'data_path': '/kaggle/input/aura-v10-data',  # Path to dataset folder
    'output_dir': '/kaggle/working',              # Output directory for models/logs
    
    # ========== Logging ==========
    'log_interval': 100,            # Log every N steps
    'save_best_model': True,        # Save model with best validation loss
}

# Print configuration summary
print("="*60)
print("🎛️  TRAINING CONFIGURATION")
print("="*60)
print(f"Model: {CONFIG['model_name']}")
print(f"Epochs: {CONFIG['epochs']}")
print(f"Effective Batch Size: {CONFIG['batch_size'] * CONFIG['gradient_accumulation']}")
print(f"Learning Rates: Encoder={CONFIG['learning_rate_encoder']}, Heads={CONFIG['learning_rate_heads']}")
print(f"Max Sequence Length: {CONFIG['max_length']} tokens")
print(f"Output Directory: {CONFIG['output_dir']}")
print("="*60)

## 3. Helper Functions

These utility functions help with:
1. **Visualization**: Plotting training curves and confusion matrices
2. **Metrics**: Computing task-specific evaluation metrics
3. **Reproducibility**: All plots use consistent styling

In [ ]:
def plot_history(history: Dict[str, List[float]], metric: str = 'loss'):
    """
    Plot training history for a given metric.
    
    Args:
        history: Dictionary with keys 'train_{metric}' and optionally 'val_{metric}'
        metric: Metric name to plot ('loss', 'accuracy', etc.)
    """
    plt.figure(figsize=(10, 6))
    
    # Plot training metric
    train_key = f'train_{metric}'
    if train_key in history:
        plt.plot(history[train_key], label='Train', marker='o', linewidth=2)
    
    # Plot validation metric if available
    val_key = f'val_{metric}'
    if val_key in history:
        plt.plot(history[val_key], label='Validation', marker='s', linewidth=2)
    
    plt.title(f'Model {metric.capitalize()} Over Time', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel(metric.capitalize(), fontsize=12)
    plt.legend(fontsize=11, loc='best')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_confusion_matrix(y_true: np.ndarray, 
                          y_pred: np.ndarray, 
                          labels: List[str], 
                          title: str = "Confusion Matrix",
                          normalize: bool = True):
    """
    Plot a confusion matrix with optional normalization.
    
    Args:
        y_true: True labels (1D array)
        y_pred: Predicted labels (1D array)
        labels: List of label names for axes
        title: Plot title
        normalize: If True, normalize by row (show percentages)
    """
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Normalize if requested
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        fmt = '.2%'  # Percentage format
        cbar_label = 'Ratio of samples'
    else:
        fmt = 'd'  # Integer format
        cbar_label = 'Number of samples'
    
    # Plot
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt=fmt, cmap='Blues', 
                xticklabels=labels, yticklabels=labels,
                cbar_kws={'label': cbar_label}, vmin=0, vmax=1 if normalize else None)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.ylabel('Actual', fontsize=12)
    plt.xlabel('Predicted', fontsize=12)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()


def compute_metrics(y_true: np.ndarray, 
                    y_pred: np.ndarray, 
                    task_name: str,
                    target_names: Optional[List[str]] = None) -> Dict[str, float]:
    """
    Compute and print comprehensive metrics for a classification task.
    
    Args:
        y_true: True labels
        y_pred: Predicted labels (for multi-label, should be binary matrix)
        task_name: Name of the task (for printing)
        target_names: Optional list of class names
    
    Returns:
        Dictionary of metric values
    """
    print(f"\n{'='*60}")
    print(f"📊 {task_name} Metrics")
    print(f"{'='*60}")
    
    # Check if multi-label (2D array)
    is_multilabel = len(y_pred.shape) == 2 and y_pred.shape[1] > 1
    
    if is_multilabel:
        # Multi-label classification (e.g., Emotions)
        print("Task Type: Multi-Label Classification\n")
        
        # Compute per-class metrics
        precision = precision_score(y_true, y_pred, average=None, zero_division=0)
        recall = recall_score(y_true, y_pred, average=None, zero_division=0)
        f1 = f1_score(y_true, y_pred, average=None, zero_division=0)
        
        # Print per-class results
        if target_names:
            print(f"{'Class':<15} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
            print("-" * 60)
            for i, name in enumerate(target_names):
                print(f"{name:<15} {precision[i]:<12.4f} {recall[i]:<12.4f} {f1[i]:<12.4f}")
        
        # Compute averages
        metrics = {
            'precision_macro': precision_score(y_true, y_pred, average='macro', zero_division=0),
            'recall_macro': recall_score(y_true, y_pred, average='macro', zero_division=0),
            'f1_macro': f1_score(y_true, y_pred, average='macro', zero_division=0),
            'precision_micro': precision_score(y_true, y_pred, average='micro', zero_division=0),
            'recall_micro': recall_score(y_true, y_pred, average='micro', zero_division=0),
            'f1_micro': f1_score(y_true, y_pred, average='micro', zero_division=0),
        }
        
        print("\nMacro-Averaged Metrics:")
        print(f"  Precision: {metrics['precision_macro']:.4f}")
        print(f"  Recall:    {metrics['recall_macro']:.4f}")
        print(f"  F1-Score:  {metrics['f1_macro']:.4f}")
        
    else:
        # Binary or multi-class classification
        print("Task Type: Binary/Multi-Class Classification\n")
        
        # Accuracy
        acc = accuracy_score(y_true, y_pred)
        print(f"Accuracy: {acc:.4f}\n")
        
        # Classification report
        print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))
        
        # Store metrics
        metrics = {
            'accuracy': acc,
            'f1_weighted': f1_score(y_true, y_pred, average='weighted', zero_division=0),
            'f1_macro': f1_score(y_true, y_pred, average='macro', zero_division=0)
        }
    
    print("=" * 60)
    return metrics


print("✅ Helper functions loaded successfully!")